# Dev notebook for updates

`Local dependence for the MPO`

In [14]:
import numpy as np
from dmrg.MPS import MPS 
# from dmrg.MPO import MPO_TFI
from dmrg.cont import CONT
from dmrg.dmrg import dmrg
from dmrg.obs import observables

import matplotlib.pyplot as plt

In [30]:
class MPO_TFI():

    Id = np.identity(2)
    Z = np.array([[1,0],[0,-1]])
    X = np.array([[0,1],[1,0]])

    def __init__(self,J,h_x,pol=None,d=2):
        self.J = J 
        self.h_x = h_x 
        self.d = d
        self.pol = pol


    def Wl(self):

        Wleft = np.zeros((2,2,3))

        Wleft[:,:,0] = MPO_TFI.Id
        Wleft[:,:,1] = -self.J*MPO_TFI.Z
        Wleft[:,:,2] = -self.h_x*MPO_TFI.X

        if self.pol == 'tot':
            Wleft[:,:,2] -= 10*MPO_TFI.Z

        return Wleft


    def mpo(self,p=None):

        MPO = np.zeros((2,2,3,3))

        MPO[:,:,0,0] = MPO_TFI.Id
        
        MPO[:,:,0,1] = -self.J* MPO_TFI.Z
        MPO[:,:,0,2] = -self.h_x*MPO_TFI.X

        MPO[:,:,1,2] = MPO_TFI.Z
        MPO[:,:,2,2] = MPO_TFI.Id 

        if p%5 == 0:
            MPO[:,:,0,2] += 10*MPO_TFI.Z

        return MPO


    def Wr(self):

        Wright = np.zeros((2,2,3))

        Wright[:,:,0] = -self.h_x*MPO_TFI.X
        Wright[:,:,1] = MPO_TFI.Z 
        Wright[:,:,2] = MPO_TFI.Id 

        if self.pol == 'tot':
            Wright[:,:,0] -= 10*MPO_TFI.Z

        return Wright
    

In [33]:
L = 40
mps = MPS(L)
h = MPO_TFI(.1,1,pol='tot')
cont = CONT(mps,h) 
sys = dmrg(cont,chi=2)
obs = observables(mps)
En0 = sys.infinite()
sys.chi = 100
print(sys.chi)
EE = []
EE.append(En0[-1])
EE_pre = []
k = 0 
print(En0)

for site,dir in mps.first_sweep():
    En,_ = sys.step2sites(site,dir=dir)
    EE.append(En)

for site,dir in mps.sweep():
    En,_ = sys.step2sites(site,dir=dir)
    EE.append(En)

Z = []
for i in range(2,L-2):
    En_fin,_ =sys.step2sites(i,dir='r',stage='Final')
    EE.append(En)
    
    # k += 1
    # if i ==L//2:
    Z += [obs.single_site(i,h.Z).real]

100
[ -22.11315128  -24.11718522  -26.12217865  -37.18186273  -48.25135725
  -50.24675904  -52.25166511  -54.25666727  -65.31647892  -76.38597345
  -78.38137524  -80.38628131  -82.39128347  -93.45109512 -104.52058965
 -106.51599144 -108.52089751 -110.52589967 -121.58571132]
